 ## CAPSTONE PROJECT
 This notebook will be used for the capstone project Applied Data Science. 
 
 #### Project Title: Thai restaurant in Switzerland 

----------------------------------------------------------------------------
----------------------------------------------------------------------------
### Introduction/Business Problem

The problem we are trying to solve: 

--> we want to open a new Thai restaurant in Switzerland once the Covid-19 crisis is over

--> the new restaurant should be in a densely populated region with strong economic activity and relatively high income 

--> the new restaurant should not be too close to already existing/popular Thai restaurants 

--> so ultimately: where in Switzerland should we open the new superb Thai restaurant?



----------------------------------------------------------------------------
----------------------------------------------------------------------------
### Data

The data (and their sources) required to answer the question: 

1) GDP per capita, from the Swiss Federal Office of Statistics:
https://www.bfs.admin.ch/bfs/de/home/statistiken/volkswirtschaft/volkswirtschaftliche-gesamtrechnung/bruttoinlandprodukt-kanton.assetdetail.10647589.html

2) Income per capita, from the Swiss Federal Office of Statistics:
https://www.bfs.admin.ch/bfs/de/home/statistiken/wirtschaftliche-soziale-situation-bevoelkerung/einkommen-verbrauch-vermoegen/haushaltsbudget.assetdetail.10867286.html


3) Also a file listing the most populous cities in Switzerland, with number of inhabitants per city, will be used. 
This information can be obtained in this site: 
https://opendata.swiss/de/group/population

The following will be used with Python in order to support the project:

4) Nominatim Geopy module to locate the coordinates of cities. 
5) Foursquare location data to identify already existing Thai restaurants in Switzerland.


Data manipulation will be required to identify possible cities to open the new restaurant, therefore first using (3) and (4) mentioned above.
Then it will be necessary to combine data from (1) and (2) to pre-select most interesting regions for such project.
Finally, we will have to exclude some cities from our scope by also using (5) to identify which existing venues would be a competition which we would like to avoid. 





----------------------------------------------------------------------------